# Tidying the results of a Google survey

We conducted a [survey](https://docs.google.com/forms/d/e/1FAIpQLSdIg3yZqSPxCac-ESLjnlfEcE5PLBo02TeBP42lgZJlUlry5w/viewform), and now we've got some results. Let's take a look. The survey responses have been deidentified and included in this repo as "data/deidentified.csv". Here I'm reading this csv in with `pandas`. The `read_csv()` function returns a `DataFrame` object. Just to get a sense of what the data look like, I print the column names and first three rows with the `head()` method available on all `DataFrame` objects.

In [1]:
import pandas
results = pandas.read_csv('data/deidentified.csv')
results.head(3)

,At what kind of facilities would you like to see the meetup?,At what skill level would you like to see events hosted?,How are you using Python?,How frequently would you be interested in attending MadPy meetings?,How would you prefer to communicate with the group online?,Is alcohol okay at meetups?,Is food okay at meetups?,What day(s) of the week work best for you?,What do you want to get out of a Madison Python meetup?,What format(s) of events would you like to see at MadPy?,What kind(s) of events would you like to see at MadPy?,What other tech meetups do you go to?,What time(s) of day work best for you?,Where in town works best for you to attend the meetup?
0,"Libraries, Offices","Introductory, Intermediate, Expert","Software (engineering/developing), Web (django...",A few times a year,"Email, IRC channel",No,Yes,"Tuesday, Thursday","Learning from others, teaching others","Presentations, Hack Nights, Python Schooling","Software (engineering/developing), Science (sc...",Madison DevOps (https://www.meetup.com/Madison...,After work hours (6pm - 9pm),"Downtown, East"
1,"Libraries, Bars, Offices, UW Facilities? If av...","Intermediate, Expert",Science (scientific & numeric computing/data s...,"A few times a year, Monthly",Email,Yes,Yes,"Monday, Tuesday, Wednesday, Thursday, Friday, ...",Learn some new concepts/features/libraries. Ne...,"Presentations, Hack Nights, Social","Software (engineering/developing), Science (sc...",NaN,"After work hours (6pm - 9pm), Night/bar time (...","Near West, Downtown, East"
2,"Libraries, Offices","Intermediate, Expert","Software (engineering/developing), Science (sc...","A few times a year, Monthly",Email,Yes,Yes,"Wednesday, Thursday",NaN,"Presentations, Hack Nights, Python Schooling","Software (engineering/developing), Science (sc...",Big Data Madison (https://www.meetup.com/BigDa...,"After work hours (6pm - 9pm), Night/bar time (...",Downtown


## Get a condensed view of all the survey questions

The header column is very verbose, including the complete question text. Let's look at the survey questions in a more condensed form. These responses have been deidentified, which means that a few questions (email, additional comments) and id fields (timestamps) have been removed.

In [2]:
questions = pandas.Series(results.columns, name='question')
# Prevent print from truncating question text
pandas.set_option('display.max_colwidth', int(questions.apply(len).max()+1))
questions

0            At what kind of facilities would you like to see the meetup?
1                At what skill level would you like to see events hosted?
2                                               How are you using Python?
3     How frequently would you be interested in attending MadPy meetings?
4              How would you prefer to communicate with the group online?
5                                             Is alcohol okay at meetups?
6                                                Is food okay at meetups?
7                              What day(s) of the week work best for you?
8                 What do you want to get out of a Madison Python meetup?
9                What format(s) of events would you like to see at MadPy?
10                 What kind(s) of events would you like to see at MadPy?
11                                  What other tech meetups do you go to?
12                                 What time(s) of day work best for you?
13                 Where in town works

## Melting the data from wide to long

The default output for Google survey responses is in "wide format" where a single person has multiple responses in each row. This is bad because it means our column headers contain data that we want to process. Really what we want is the survey responses in "long format", where each row is a single response, and then each person has multiple rows. To learn more about the benefits of wide versus long format, check out Hadley Wickham's paper ["Tidy Data"](https://www.jstatsoft.org/article/view/v059i10).

To do the transformation, the first thing we need to do is assign a name for each person. This will help us keep responses together later on.

In [3]:
# Give people names p0 ... pN
names = ['p{}'.format(i) for i in list(results.index)]
results.insert(0, 'person', names)  # cf. results['person'] = names
results.head(2)

,person,At what kind of facilities would you like to see the meetup?,At what skill level would you like to see events hosted?,How are you using Python?,How frequently would you be interested in attending MadPy meetings?,How would you prefer to communicate with the group online?,Is alcohol okay at meetups?,Is food okay at meetups?,What day(s) of the week work best for you?,What do you want to get out of a Madison Python meetup?,What format(s) of events would you like to see at MadPy?,What kind(s) of events would you like to see at MadPy?,What other tech meetups do you go to?,What time(s) of day work best for you?,Where in town works best for you to attend the meetup?
0,p0,"Libraries, Offices","Introductory, Intermediate, Expert","Software (engineering/developing), Web (django/flask/other web d...",A few times a year,"Email, IRC channel",No,Yes,"Tuesday, Thursday","Learning from others, teaching others","Presentations, Hack Nights, Python Schooling","Software (engineering/developing), Science (scientific & numeric...","Madison DevOps (https://www.meetup.com/Madison-Devops), Mad-Rail...",After work hours (6pm - 9pm),"Downtown, East"
1,p1,"Libraries, Bars, Offices, UW Facilities? If available to non-stu...","Intermediate, Expert",Science (scientific & numeric computing/data science),"A few times a year, Monthly",Email,Yes,Yes,"Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday",Learn some new concepts/features/libraries. Network. Collaborate...,"Presentations, Hack Nights, Social","Software (engineering/developing), Science (scientific & numeric...",NaN,"After work hours (6pm - 9pm), Night/bar time (9pm - 2am)","Near West, Downtown, East"


Now we can transform the data from wide to long format using the `pandas.melt()` function. The first argument is the DataFrame we are melting. Then we provide the "id_vars". These are the variables that will become duplicated in the melted DataFrame. For survey responses, the id variable is "person", meaning each person will go from having responses in one row to having responses in multiple rows.

The other variables ("var_name" and "value_name") are optional, they just provide the names for the resulting columns. "var_name" is the name for the new column containing the names of the old columns. In our case, the column headers are question texts, so we set this to "question". The "value_name" is the name of the data in the cell of the original DataFrame. In our case, the data are responses. For some questions (e.g. questions with checkboxes) these responses are actually multiple responses joined together with a comma. We'll split them later. For now, let's call the value column "response_str" to remind ourselves that we aren't yet to a single response per row.

In [4]:
response_strs = pandas.melt(results, 'person', var_name='question', value_name='response_str')
response_strs.head(3)

,person,question,response_str
0,p0,At what kind of facilities would you like to see the meetup?,"Libraries, Offices"
1,p1,At what kind of facilities would you like to see the meetup?,"Libraries, Bars, Offices, UW Facilities? If available to non-stu..."
2,p2,At what kind of facilities would you like to see the meetup?,"Libraries, Offices"


Another helpful function on DataFrames is `sort_values()`. After the melt, by default responses are grouped by question. Here I'm using the `sort_values()` function to rearrange the DataFrame by person to show how now each person's responses are spit across multiple rows after the split.

In [5]:
response_strs.sort_values('person').head(3)

,person,question,response_str
0,p0,At what kind of facilities would you like to see the meetup?,"Libraries, Offices"
303,p0,How frequently would you be interested in attending MadPy meetings?,A few times a year
404,p0,How would you prefer to communicate with the group online?,"Email, IRC channel"


## Splitting responses

The last step is to split "response_str" into multiple responses. Basically we are melting the DataFrame again, this time on the number response for each question. But we can't use the `melt()` function because the multiple responses are contained within a single value (not split across multiple columns), and there are a variable number of responses within each response string. To help me do this, I'm going to write a custom melt function that accepts a DataFrame containing the response strings, and returns a longer pandas.DataFrame of responses--one per row.

In [6]:
def melt_response_strs(response_strs):
    """Convert a DataFrame with response strings to a DataFrame with one response per row.
    
    An example response str is 'Libraries, Offices'. The resulting DataFrame will have
    two rows, one for each response item, e.g, ['Libraries', 'Offices'].
    """
    melted_rows = []
    for row in response_strs.itertuples():
        try:
            responses = [response.strip() for response in row.response_str.split(',')]
        except AttributeError:
            # If the response str can't be split, e.g., they didn't answer this question,
            # put it as a single value in a list.
            responses = [row.response_str]
        
        # Create a new row for each response
        for response_n, response in enumerate(responses):
            melted_rows.append(dict(person=row.person, question=row.question,
                                    response_n=response_n, response=response))

    return pandas.DataFrame.from_records(melted_rows,
                                         columns=['person', 'question', 'response_n', 'response'])

responses = melt_response_strs(response_strs)
responses.head()

,person,question,response_n,response
0,p0,At what kind of facilities would you like to see the meetup?,0,Libraries
1,p0,At what kind of facilities would you like to see the meetup?,1,Offices
2,p1,At what kind of facilities would you like to see the meetup?,0,Libraries
3,p1,At what kind of facilities would you like to see the meetup?,1,Bars
4,p1,At what kind of facilities would you like to see the meetup?,2,Offices


## Next step: identifying different types of responses

Right now we don't know anything about the type of question being answered. For example, some questions only allow one choice (radio button), while others allow free responses, e.g., an "Other:" choice. To properly understand the results of the survey it's helpful to know what the options were on any given question.

You can easily keep a record of all choices for all survey questions by hand, but you know python. Rather than duplicating information (and keeping track of it by hand), why not just scrape the relevant information from the survey html? With python, it's not that hard.

In [7]:
responses.to_csv('data/tidied.csv', index=False)